Введём функцию поиска минимума методом деформированного многогранника

In [1]:
import numpy as np

def deform(f, x, maxiter):
    n = len(x)
    sim = np.ndarray(shape=(n + 1, n), dtype=float)
    sim[n] = [i for i in x]
    for i in range(n):     #создаём симплекс
        sim[i][i] = 1
    print(step(f, sim, n, 0, maxiter))
    
def step(f, sim, n, iter, maxiter): 
    iter += 1
    alf = 1
    bet = 0.5
    gam = 2
    delt = 0.5
    w = g = b = sj = sjj = 0
    fb = 1e+307
    fw = fg = -1e+307
    for i in range(n + 1):     #находим три рабочие точки
        if fg < f(sim[i]):
            if fw < f(sim[i]):
                g = w
                fg = fw
                w = i
                fw = f(sim[i])
            else:
                g = i
                fg = f(sim[i])
        if fb > f(sim[i]):
            fb = f(sim[i])
            b = i
    c = np.array([0] * n)
    c = c * 1.0
    for i in range(n):     #находим центр тяжести
        l = [sim[k][i] for k in range(n + 1)]
        l[w] = 0
        c[i] = sum(l)
        c[i] /= n
    xr = np.array([0] * n)     #отражение
    xr = c + alf * (c - sim[w])
    fr = f(xr)
    if fr < fb:     #растяжение
        xe = np.array([0] * n)
        xe = c + gam * (xr - c)
        if f(xe) < fr:
            sim[w] = xe
        else:
            sim[w] = xr
    else:        
        if fr < fg:
            sim[w] = xr
        if fg < fr and fr <= fw:
            sim[w] = xr
            fw = fr
            sj = 1
        if fw < fr:
            sj = 1
        
        if sj == 1:    #сжатие
            xs = c + bet * (sim[w] - c)
            if f(xs) > fw:
                sjj = 1
        
        if sjj == 1:        #гомотетия
            for i in range(n + 1):
                sim[i] = sim[b] + delt * (sim[i] - sim[b])

    if iter < maxiter:
        return step(f, sim, n, iter, maxiter)
    else:
        return sim[b], fb

Проверим её на функции, экстремумом которой является точка 1, 4. Значение в этой точке - 21. Движение начнём из точки (0,1) координат, сделаем 5 итерации

In [5]:
def f(k):
    return k[0] ** 2 + k[1] * k[0] + k[1] ** 2 - 6 * k[0] - 9 * k[1]

k = [0,1]
deform(f,k,5)

(array([ 1.,  5.]), -20.0)


Повысим точность. Сделаем 10 иттераций. Начать при этом можно из точки (1,1)

In [4]:
k = [1,0]
deform(f,k,10)

(array([ 1.,  4.]), -21.0)


Видно, что при увеличении числа итераций растёт точность нахождения минимума, причём при 10 точность выше эпсилон